In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import imp
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri

RANDOM_SEED = 0
numpy2ri.activate()
ro.r('set.seed({})'.format(RANDOM_SEED))

import fairtest.utils.log as fairtest_log
imp.reload(fairtest_log)
fairtest_log.set_params(filename='fairtest.log')

In [31]:
import fairtest.investigation as inv
import fairtest.testing as testing
import fairtest.discovery as discovery
import fairtest.error_profiling as error_profiling
import fairtest.modules.metrics as metrics
import fairtest.modules.metrics.correlation as correlation
import fairtest.modules.metrics.regression as regression
import fairtest.modules.metrics.binary_metrics as binary_metrics
import fairtest.modules.statistics.confidence_interval as intervals
import ast
import sklearn.preprocessing as preprocessing
imp.reload(intervals)
imp.reload(inv)
imp.reload(testing)
imp.reload(discovery)
imp.reload(error_profiling)
imp.reload(metrics)
imp.reload(correlation)
imp.reload(binary_metrics)
imp.reload(regression)

'''
dataname = 'adult'
data = pd.read_csv(
            '../data/adult/adult.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
data = data.drop('fnlwgt', axis=1)
#data['race'] = map(lambda x: 'Asian' if 'Asian' in x else 'Indian' if 'Indian' in x else x, data['race'])
#data['race'] = map(lambda x: 'Asian' if 'Asian' in x else 'Black & Other' if ('Indian' in x or 'Other' in x or 'Black' in x) else x, data['race'])
TARGET = 'income'
SENS = ['sex', 'race']
EXPL = []
'''

'''
dataname = 'medical_bin'
data = pd.read_csv(
            '../data/medical/predictions_bin.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'Prediction'
GROUND_TRUTH = 'Ground_Truth'
SENS = ['Age']
EXPL = []
'''
'''
dataname = 'medical_reg'
data = pd.read_csv(
            '../data/medical/predictions_reg_log.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'Prediction'
GROUND_TRUTH = 'Ground_Truth'
SENS = ['Age']
EXPL = []
'''
'''
dataname = 'berkeley'
data = pd.read_csv(
            '../data/berkeley/berkeley.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'accepted'
SENS = ['gender']
EXPL = ['department']
#EXPL = []
'''
'''
dataname = 'staples'
data = pd.read_csv(
            '../data/staples/staples.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
data = data.drop('zipcode', axis=1)
data = data.drop('distance', axis=1)
data = data.drop('city', axis=1)
TARGET = 'price'
SENS = ['income', 'race']
EXPL = []
'''
'''
data['race'] = map(lambda x: 'Asian' if 'Asian' in x else 
                   'Indian & Alaska Native' if 'Indian' in x  else
                   'Black' if 'Black' in x  else
                   'Pacific Islander' if 'Pacific' in x else
                   'White' if 'White' in x else
                   'Hispanic' if 'Latino' in x else
                   '2+ Races' if 'Two' in x else
                   'Other' if 'Other' in x else
                    x, data['race'])
'''
'''
dataname = 'overfeat'
data = pd.read_csv(
            '../data/images/overfeat_raw.txt',
            header=0,
            sep=r'\s*\t\s*',
            engine='python',
            na_values="?")
TARGET = 'Labels'
SENS = ['Race']
EXPL = []
labeled_data = map(lambda s: ast.literal_eval(s), data[TARGET])
for l in labeled_data:
    assert len(l) == 5
label_encoder = preprocessing.MultiLabelBinarizer()
labeled_data = label_encoder.fit_transform(labeled_data)
labels = label_encoder.classes_
df_labels = pd.DataFrame(labeled_data, columns=labels)
data = pd.concat([data.drop(TARGET, axis=1), df_labels], axis=1)
TARGET = labels.tolist()
'''

dataname = 'caffe'
data = pd.read_csv(
            '../data/images/caffe.txt',
            header=0,
            sep=r'\s*\t\s*',
            engine='python',
            na_values="?")
TARGET = 'Labels'
SENS = ['Race']
EXPL = []
labeled_data = map(lambda s: ast.literal_eval(s), data[TARGET])
for l in labeled_data:
    assert len(l) == 5
label_encoder = preprocessing.MultiLabelBinarizer()
labeled_data = label_encoder.fit_transform(labeled_data)
labels = label_encoder.classes_
df_labels = pd.DataFrame(labeled_data, columns=labels)
data = pd.concat([data.drop(TARGET, axis=1), df_labels], axis=1)
TARGET = labels.tolist()

'''
dataname = 'benchmark'
data = pd.read_csv(
            '../data/benchmark/benchmark.csv',
            header=0,
            sep=r'\s*,\s*',
            engine='python',
            na_values="?")
TARGET = 'price'
SENS = ['race', 'income']
EXPL = []
'''

#metrics = {'gender': 'Diff'}
metrics = {}

#exp = testing.Testing(data, SENS, TARGET, EXPL, metrics=metrics, train_size=0.5, random_state=RANDOM_SEED)
#exp = error_profiling.ErrorProfiling(data, SENS, TARGET, GROUND_TRUTH, EXPL, metrics=metrics, train_size=0.5, random_state=RANDOM_SEED)
exp = discovery.Discovery(data, SENS, TARGET, EXPL, metrics=metrics, train_size=0.5, topk=30, random_state=RANDOM_SEED)
print len(exp.train_set), len(exp.test_set)
exp.train_set.tail()

INFO:root:New Discovery Investigation
INFO:root:Encoding Feature Race
INFO:root:Target Feature: Target(names=[accordion ... zebra], arity=None)
INFO:root:Training Size 1307 -  Testing Size 1308
INFO:root:Choosing metric REGRESSION for feature Race


1307 1308


,Race,accordion,airplane,ant,antelope,apple,armadillo,artichoke,axe,baby bed,...,vacuum,violin,volleyball,waffle iron,washer,water bottle,watercraft,whale,wine bottle,zebra
1731,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
763,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1653,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2607,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [32]:
import fairtest.modules.context_discovery.guided_tree as guided_tree
imp.reload(guided_tree)
inv.train([exp], score_aggregation='avg', min_leaf_size=100, max_depth=5)

INFO:root:Begin training phase with protected feature Race
INFO:root:Building a Guided Decision Tree


In [6]:
print exp.trained_trees['Age']

KeyError: 'Age'

In [33]:
import fairtest.modules.statistics.hypothesis_test as tests
import fairtest.modules.statistics.multiple_testing as multitest
import fairtest.modules.context_discovery.tree_parser as tree_parser
import fairtest.modules.statistics.confidence_interval as intervals
imp.reload(tests)
imp.reload(multitest)
imp.reload(tree_parser)
imp.reload(intervals)

inv.test([exp], exact=True, prune_insignificant=True)

INFO:root:Begin testing phase
INFO:root:Testing 30 hypotheses


In [34]:
import fairtest.modules.bug_report.report as rep
import fairtest.modules.bug_report.filter_rank as fr
imp.reload(rep)
imp.reload(fr)

output_dir = '../results_new'
inv.report([exp], dataname, output_dir=None, filter_conf=0.95, node_filter=fr.FILTER_BETTER_THAN_ANCESTORS)

Commit Hash: 	82c74056ed3f5b5cb19f5eb4179738c81cd0f9f2

Dataset: caffe
Train Size: 1307
Test Size: 1308
S: ['Race']
X: []
E: None
O: [accordion ... zebra]

Train Params: 	{'max_bins': 10, 'min_leaf_size': 100, 'max_depth': 5, 'agg_type': 'avg'}
Test Params: 	{'prune_insignificant': True, 'exact': True, 'family_conf': 0.95}
Report Params: 	{'node_filter': 'better_than_ancestors'}

Report of associations of O=[accordion ... zebra] on Si = Race:
Association metric: REG(topk=30)

Global Population 0 of size 1308

Average Effect of top-30 labels: 0.00548725955171

                ci_low   ci_high          pval
face powder  -0.133316 -0.053102  8.221013e-12
maillot      -0.158287 -0.058236  2.952303e-10
cart          0.014096  0.064829  2.808148e-05
drum          0.009847  0.060010  3.228987e-04
helmet        0.010112  0.088201  1.962883e-03
cattle        0.003931  0.042884  3.944134e-03
person       -0.055942 -0.004605  5.012821e-03
lipstick     -0.061795 -0.003847  8.482389e-03
train      

[{}]